In [1]:
import os
import sys
import glob
import shutil

sys.path.append('..')
from preprocessing.ct_batch import CTImagesBatch
from dataset.dataset.dsindex import FilesIndex
print('OK')

OK


### Global index

In [2]:
DIR_TEST = '/notebooks/data/MRT/nci/*'
DIR_DUMP = '/notebooks/Koryagin/Batch_dev/dir_for_dumps/'

In [3]:
if os.path.exists(DIR_DUMP):
    shutil.rmtree(DIR_DUMP)

In [4]:
ind = FilesIndex(path=DIR_TEST, dirs=True)

### Create batch by subindexing

In [5]:
batch = CTImagesBatch(ind.create_subset(ind.index[[2,5,7]]))

In [6]:
batch.indices

array(['274a81c75d244187247789bd71de2b3a',
       '645e7f46eb9b834153ecf8e2b2921fe5',
       'd73264d2c4f71ef6bf8593ac55565d93'], 
      dtype='<U56')

### Test actions

* check load

In [7]:
batch.load(fmt='dicom')

['274a81c75d244187247789bd71de2b3a'] {}
274a81c75d244187247789bd71de2b3a['645e7f46eb9b834153ecf8e2b2921fe5']
 {}
645e7f46eb9b834153ecf8e2b2921fe5
['d73264d2c4f71ef6bf8593ac55565d93'] {}
d73264d2c4f71ef6bf8593ac55565d93
[[[-1024 -1024 -1024 ..., -1024 -1024 -1024]
  [-1024 -1024 -1024 ..., -1024 -1024 -1024]
  [-1024 -1024 -1024 ..., -1024 -1024 -1024]
  ..., 
  [-1024 -1024 -1024 ..., -1024 -1024 -1024]
  [-1024 -1024 -1024 ..., -1024 -1024 -1024]
  [-1024 -1024 -1024 ..., -1024 -1024 -1024]]

 [[-1024 -1024 -1024 ..., -1024 -1024 -1024]
  [-1024 -1024 -1024 ..., -1024 -1024 -1024]
  [-1024 -1024 -1024 ..., -1024 -1024 -1024]
  ..., 
  [-1024 -1024 -1024 ..., -1024 -1024 -1024]
  [-1024 -1024 -1024 ..., -1024 -1024 -1024]
  [-1024 -1024 -1024 ..., -1024 -1024 -1024]]

 [[-1024 -1024 -1024 ..., -1024 -1024 -1024]
  [-1024 -1024 -1024 ..., -1024 -1024 -1024]
  [-1024 -1024 -1024 ..., -1024 -1024 -1024]
  ..., 
  [-1024 -1024 -1024 ..., -1024 -1024 -1024]
  [-1024 -1024 -1024 ..., -1024 -

In [9]:
batch_copy = deepcopy(batch)

* check attrs

In [10]:
batch._data.shape

(471, 512, 512)

In [7]:
batch._bounds

array([  0, 184, 314, 471])

* check resize, segment, dump

In [8]:
batch2 = batch.resize(shape=(64, 64, 32)) #, n_workers=4)

[] {'shape': (64, 64, 32), 'res': array([[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       ..., 
       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
     

In [13]:
batch._data.shape

(96, 64, 64)

In [14]:
batch._bounds

array([ 0, 32, 64, 96])

* check dump

In [9]:
batch2.dump(dst=DIR_DUMP)

### Build batch from dump and compare

In [30]:
ind_dumped = FilesIndex(path=DIR_DUMP + '/*', dirs=True)

In [31]:
ind_dumped.index

array(['274a81c75d244187247789bd71de2b3a',
       '645e7f46eb9b834153ecf8e2b2921fe5',
       'd73264d2c4f71ef6bf8593ac55565d93'], 
      dtype='<U76')

In [32]:
batch_from_dump = CTImagesBatch(ind_dumped)

In [33]:
batch_from_dump.load(fmt='blosc', src=None)

<class 'list'>
[array([[[-1003., -1001.,  -985., ...,  -976.,  -999., -1007.],
        [-1019., -1008., -1006., ...,  -999., -1023., -1024.],
        [ -983.,  -999., -1007., ..., -1003., -1021., -1016.],
        ..., 
        [ -849.,  -918.,  -929., ...,  -940.,  -955.,  -964.],
        [ -900.,  -920.,  -897., ...,  -981.,  -960.,  -946.],
        [ -916.,  -894.,  -905., ...,  -803.,   108.,  -518.]],

       [[ -998., -1008., -1008., ...,  -998., -1016.,  -991.],
        [ -987., -1021.,  -987., ...,  -993., -1013.,  -967.],
        [-1004.,  -996., -1002., ..., -1008., -1007.,  -986.],
        ..., 
        [ -910.,  -905.,  -934., ...,  -973.,  -962.,  -953.],
        [ -923.,  -895.,  -898., ...,  -969.,  -974.,  -951.],
        [ -942.,  -913.,  -926., ...,  -860.,    68.,  -435.]],

       [[-1018.,  -989., -1013., ...,  -979., -1007.,  -984.],
        [-1005.,  -984., -1012., ..., -1002.,  -982.,  -996.],
        [-1013.,  -991.,  -984., ...,  -986., -1018., -1013.],
       

In [34]:
def plot_slices(height, num_pers, *batches):
    fig, axes = plt.subplots(1, len(batches), figsize = (10, len(batches)*8))
    com_args = dict(clim=(-1200, 300), cmap=(plt.cm.gray))
    
    for i in range(len(batches)):
        axes[i].imshow(batches[i].get_axial_slice(num_pers, height), **com_args)
    
    plt.show()

In [36]:
batch.indices

array(['274a81c75d244187247789bd71de2b3a',
       '645e7f46eb9b834153ecf8e2b2921fe5',
       'd73264d2c4f71ef6bf8593ac55565d93'], 
      dtype='<U56')

In [37]:
batch_from_dump.indices

array(['274a81c75d244187247789bd71de2b3a',
       '645e7f46eb9b834153ecf8e2b2921fe5',
       'd73264d2c4f71ef6bf8593ac55565d93'], 
      dtype='<U76')

In [38]:
interact(lambda height: plot_slices(height, 1, batch_copy, batch, batch_from_dump), 
         height=(0.01, 0.99, 0.01))

<function __main__.<lambda>>

### XIP

In [ ]:
xip = batch.make_xip()

In [ ]:
interact(lambda height: plot_slices(height, 2, batch_from_dump, xip), height=(0.01, 0.99, 0.01))

In [ ]:
xip.indices

In [ ]:
xip.index.index